In [8]:
import tensorflow as tf
import numpy as np

In [9]:
# define a simple convolutional layer
def conv_layer(input, in_channels, out_channels):
    w = tf.Variable(tf.zeros([5, 5, in_channels, out_channels]))
    b = tf.Variable(tf.zeros([out_channels]))
    conv = tf.nn.conv2d(input, w, strides = [1, 1, 1, 1], padding = "SAME")
    act = tf.nn.relu(conv + b)
    return act

In [10]:
# add a fully connected layer
def fc_layer(input, in_size, out_size , act = tf.nn.relu):
    w = tf.Variable(tf.random_normal([in_size, out_size]))
    b = tf.Variable(tf.zeros([out_size]))
    sum = tf.matmul(input, w) + b
    if act == tf.nn.relu:
        out = tf.nn.relu(sum)
    else:
        out = act(sum)
    return out

In [11]:
### feed-forward setup
# setup placeholders, and reshape the data
x = tf.placeholder(tf.float32, shape = [None, 784])
y = tf.placeholder(tf.float32, shape = [None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1]) 
#四维对应：【图片的数量，图片的宽，图片的高，输入图片的厚度（黑白为1，RGB为3）】

In [12]:
# create the network
conv1 = conv_layer(x_image, 1, 32)
pool1 = tf.nn.max_pool(conv1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")
conv2 = conv_layer(pool1, 32, 64)
pool2 = tf.nn.max_pool(conv2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")
flattened = tf.reshape(pool2, [-1, 7 * 7 * 64]) #卷积层步长1，故大小不变，池化层缩水一半
fc1 = fc_layer(flattened, 7 * 7 * 64, 1024)
logits = fc_layer(fc1, 1024, 10)
logits

<tf.Tensor 'Relu_7:0' shape=(?, 10) dtype=float32>

In [13]:
## loss and training
# compute cross entropy as our loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))
# use an adamOptimizer to train the network
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# compute the accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy

<tf.Tensor 'Mean_3:0' shape=() dtype=float32>

In [14]:
import tensorflow.examples.tutorials.mnist.input_data as input_data

In [15]:
## train the model
#input_data = tf.examples.tutorials.mnist.input_data
mnist = input_data.read_data_sets("MINST_data/", one_hot = True)

with tf.Session() as sess:
    # initialize all the variables
    sess.run(tf.global_variables_initializer())
    # train
    for i in range(2000):
        batch = mnist.train.next_batch(100)
        if i % 500 == 0:
            [train_accuracy] = sess.run([accuracy], feed_dict = {x: batch[0], y: batch[1]})
            print("step %d, training accuracy %g" %(i, train_accuracy))
        sess.run(train_step, feed_dict = {x: batch[0], y: batch[1]})

Extracting MINST_data/train-images-idx3-ubyte.gz
Extracting MINST_data/train-labels-idx1-ubyte.gz
Extracting MINST_data/t10k-images-idx3-ubyte.gz
Extracting MINST_data/t10k-labels-idx1-ubyte.gz


AttributeError: 'module' object has no attribute 'global_variables_initializer'